# Deploy trained model to Endpoint

Endpoints provide a unified interface to invoke and manage model deployments across compute types [What are endpoints and deployments?](https://learn.microsoft.com/en-us/azure/machine-learning/concept-endpoints?view=azureml-api-2)


## Import Libraries

In [7]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Model, Environment, CodeConfiguration, OnlineRequestSettings
from azure.identity import DefaultAzureCredential
from azure.core.exceptions import ResourceNotFoundError

## Define constants

In [ ]:
# Deployment
online_endpoint_name = 'SRN-endpoint-demo1234'
registered_model_name = 'SegResNet'
deployment_name = 'blue'
inference_env_name = 'inference-env'

## Create ML Client

In [2]:
credential = DefaultAzureCredential()

ml_client = MLClient.from_config(credential=credential)

Found the config file in: ./config.json


## Get latest model to deply from registry

In [4]:
model = ml_client.models.get(name=registered_model_name, label='latest')

## Get Existing or Create Endpoint

In [25]:
try:
    endpoint = ml_client.online_endpoints.get(online_endpoint_name)
    print("Found existing endpoint.")
except ResourceNotFoundError:
    print("Creating a new endpoint...")
    endpoint = ManagedOnlineEndpoint(
        name=online_endpoint_name,
        description="An online endpoint to generate predictions",
        auth_mode="key",
        tags={"foo": "bar"},
    )

    ml_client.begin_create_or_update(endpoint)

## Get Existing or Create Scoring Environment

In [9]:
try:
    scoring_environment = ml_client.environments.get(inference_env_name, label='latest')
    print("Found existing environment.")
except ResourceNotFoundError:
    print("Creating a new environment...")
    scoring_environment = Environment(
        image="mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu20.04:latest",
        conda_file="./scoring-env.yaml",
        name=inference_env_name,
        description="Brain tumor segmentation inference environment")

    ml_client.environments.create_or_update(scoring_environment)

Found existing environment.


## Get Existing or Create Model Deployment

In [24]:
try:
    blue_deployment = ml_client.online_deployments.get(deployment_name, endpoint.name)
    print("Found existing deployment.")
except ResourceNotFoundError:
    print("Creating a new deployment...")
    blue_deployment = ManagedOnlineDeployment(
        name=deployment_name,
        endpoint_name=online_endpoint_name,
        model=model,
        environment=inference_env_name + ":1",
        code_configuration=CodeConfiguration(
            code="./",
            scoring_script="score.py"
        ),
        instance_type="Standard_E48s_v3",
        instance_count=3,
        request_settings= OnlineRequestSettings(request_timeout_ms = 90000),
    )

    deployment = ml_client.begin_create_or_update(blue_deployment)

Found existing deployment.
